In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


CNN(Convolutional Neural Network) - 합성곱 신경망

In [3]:
# MNIST 데이터를 내려받아 학습 데이터 및 테스트 데이터로 분리해서 저장한다.
# MNIST 데이터는 28 * 28의 픽셀 데이터이고 각 픽셀은 0부터 255까지의 그레이스케일을 가지고 있다.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [4]:
# 6만 개의 학습 데이터 중 1만 개의 학습 데이터를 검증 데이터로 저장한다.
# 학습 중간마다 검증 데이터로 모델의 성능을 측정하면 모델 학습이 제대로 진행되는지 검증 정확도로 확인을 할 수 있고 학습 
# 정확도는 증가하는데 검증 정확도가 증가하지 않거나 떨어질 때 조기 종료를 구현할 수 있다.
x_val = x_train[50000:]   # 학습에 사용할 6만개의 데이터 중 검증 데이터 1만개
x_train = x_train[:50000] # 학습에 사용할 6만개의 데이터 중 학습 데이터 5만개
y_val = y_train[50000:]   # 학습에 사용할 6만개의 데이터 중 검증 데이터의 레이블 1만개
y_train = y_train[:50000] # 학습에 사용할 6만개의 데이터 중 검증 데이터의 레이블 5만개
# x_train.shape

In [5]:
print('학습 데이터 => {}개'.format(x_train.shape[0]))
print('학습 데이터는 {} * {} 픽셀의 이미지'.format(x_train.shape[1], x_train.shape[2]))
print('검증 데이터 => {}개'.format(x_val.shape[0]))
print('검증 데이터는 {} * {} 픽셀의 이미지'.format(x_val.shape[1], x_val.shape[2]))
print('=' * 80)
print('x_train.shape: {}'.format(x_train.shape)) # 학습 데이터는 5만 개 이고 28 * 28 픽셀의 이미지이다.
print('x_val.shape: {}'.format(x_val.shape))     # 검증 데이터는 1만 개 이고 28 * 28 픽셀의 이미지이다.
print('x_test.shape: {}'.format(x_test.shape))   # 테스트 데이터는 1만 개 이고 28 * 28 픽셀의 이미지이다.
print('y_train.shape: {}'.format(y_train.shape)) # 학습 데이터 5만 개의 실제값(레이블, 클래스)
print('y_val.shape: {}'.format(y_val.shape))     # 검증 데이터 1만 개의 실제값
print('y_test.shape: {}'.format(y_test.shape))   # 테스트 데이터 1만 개의 실제값

학습 데이터 => 50000개
학습 데이터는 28 * 28 픽셀의 이미지
검증 데이터 => 10000개
검증 데이터는 28 * 28 픽셀의 이미지
x_train.shape: (50000, 28, 28)
x_val.shape: (10000, 28, 28)
x_test.shape: (10000, 28, 28)
y_train.shape: (50000,)
y_val.shape: (10000,)
y_test.shape: (10000,)


In [6]:
print(y_train[:10])
for i in x_train[4]:
    for j in i:
        print('%3d ' % j, end='')
    print()

[5 0 4 1 9 2 1 3 1 4]
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0  55 148 210 253 253 113  87 148  55   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0  87 232 252 253 189 210 252 25

데이터 구조를 변경한다.

In [8]:
# 28 * 28 픽셀의 단색 이미지이므로 데이터 형태를 28 * 28 * 1로 맞춰 CNN 모델에 입력하기 위해 데이터셋을 생성한다.
import numpy as np
x_train = np.reshape(x_train, (50000, 28, 28, 1))
print('x_train.shape: {}'.format(x_train.shape))
x_val = np.reshape(x_val, (10000, 28, 28, 1))
print('x_val.shape: {}'.format(x_val.shape))
x_test = np.reshape(x_test, (10000, 28, 28, 1))
print('x_test.shape: {}'.format(x_test.shape))

x_train.shape: (50000, 28, 28, 1)
x_val.shape: (10000, 28, 28, 1)
x_test.shape: (10000, 28, 28, 1)


데이터 정규화

In [9]:
# 데이터 정규화는 학습 시간을 단축하고, 더 나은 성능을 발휘하도록 도와준다.
# MNIST 데이터의 모든 값은 0부터 255의 범위 안에 있으므로 255로 나눠줌으로써 모든 값을 0부터 1 사이의 값으로 정규화 한다.
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
gray_scale = 255
x_train /= gray_scale
x_val /= gray_scale
x_test /= gray_scale

one hot encoding

In [10]:
# 손실 함수에서 크로스 엔트로피를 계산하기 위해, 실제값을 one hot encoding 값으로 변경한다.
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

텐스플로우로 CNN 구현하기